Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [34]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [9]:
URL = 'https://habr.com/ru/all/'
params = {
    'article': 'post_pos_preview'
}

In [28]:
res = requests.get(URL, params)

In [27]:
soup = BeautifulSoup(res.text, 'html.parser')

In [388]:
KEYWORDS = ['python', 'парсинг']
news_blocks = soup.find_all('article', class_='post')
#news_blocks

In [104]:
habr = pd.DataFrame()

In [105]:
for post in news_blocks:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
   
    hubs = post.find_all('div', class_='post__text')
    for hub in hubs:
            time.sleep(0.33)
            hub_lower = hub.text.lower()
            for word in KEYWORDS:
                if (word in hub_lower) or (word in post.find('a', class_='post__title_link').text.lower()):
            
                    title_element = post.find('a', class_='post__title_link')
                    data = post.find('span', class_ = 'post__time')

                    row = {'date': data.text, 'title': title_element.text, 'link': title_element.attrs.get('href')}
                    habr = pd.concat([habr, pd.DataFrame([row])])  
                    

In [106]:
habr


,date,title,link
0,вчера в 22:51,Тесты на pytest с генерацией отчетов в Allure ...,https://habr.com/ru/post/513432/
0,вчера в 09:51,Как сделать поиск по файлоболотам в 104 строки...,https://habr.com/ru/post/513372/


Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [385]:
habr_2 = pd.DataFrame()

In [386]:
for post in news_blocks:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
   
    hubs = post.find_all('div', class_='post__text')
    for hub in hubs:
            time.sleep(0.33)
            hub_lower = hub.text.lower()
            link = post.find('a', class_='post__title_link').attrs.get('href')
            
            res = requests.get(link, {'div': 'post__wrapper'})
            soup = BeautifulSoup(res.text, 'html.parser')
            full_post = soup.find('div', class_='post__wrapper')
            
            title_element = full_post.find('span', class_='post__title-text')
            date = full_post.find('span', class_ = 'post__time') 
            text = full_post.find('div', class_ = 'post__text')
        
                        
            for word in KEYWORDS:
             
                if (word in title_element.text.lower()) or (word in text.text.lower()):
                    
                    row = {'date': date.text, 'title': title_element.text, 'link': link, 'text': text.text}
                    
                    habr_2 = pd.concat([habr_2, pd.DataFrame([row])]) 
 

In [387]:
habr_2

,date,title,link,text
0,вчера в 22:51,Тесты на pytest с генерацией отчетов в Allure ...,https://habr.com/ru/post/513432/,Этот текст предназначен для начинающих тестиро...
0,вчера в 18:03,"HackTheBox. Прохождение Oouch. OAuth2, RCE в u...",https://habr.com/ru/post/513378/,\r\nПродолжаю публикацию решений отправленных ...
0,вчера в 12:12,Как видеоигры помогают прокачивать реальные на...,https://habr.com/ru/company/ruvds/blog/513382/,\n\r\n14 июля в нашем инстаграм-аккаунте и юту...
0,вчера в 09:51,Как сделать поиск по файлоболотам в 104 строки...,https://habr.com/ru/post/513372/,"Продолжая тематику коротких полезных скриптов,..."


Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [373]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
final_df = pd.DataFrame()

for email_ in EMAIL:
    params = {"email": email_}
    res = requests.post("https://digibody.avast.com/v1/web/leaks", json = params)

    df = pd.DataFrame(res.json()['value'])
    for i in range(len(df)):

        d_l = dict(df.leak_info[i])

        row = {'username': df.username[i], 'leak_date': df.leak_date[i], 'source': d_l['title'], 'description': d_l['description']}

        final_df = pd.concat([final_df, pd.DataFrame([row])]) 
final_df.reset_index(inplace = True)
final_df.drop(columns = ['index'], axis=1, inplace=True)        
final_df

,username,leak_date,source,description
0,xxx@x.ru,1513900800000,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
1,xxx@x.ru,1549411200000,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
2,aleksey1688,1485820800000,CD Projekt Red,"In March 2016, CDProjektRed.com.com's forum da..."
3,xixixixixixixixixixi,1487030400000,Parapa,"In July and August 2016, two criminals execute..."
4,xxx@x.ru,1488931200000,Sensitive Source,This source has been marked as sensitive due t...
5,xxx@x.ru,1549411200000,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
6,xxx@x.ru,1549411200000,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."
7,stalkera116,1487030400000,CFire Mail,"In July and August of 2016, two criminals carr..."
8,xxx@x.ru,1477008000000,Adobe Systems,"In October of 2013, criminals penetrated Adobe..."
9,xxx@x.ru,1548720000000,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
